In [1]:
! wget -P ../../data http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-11-07 15:41:56--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘../../data/ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.15MB/s    in 0.8s    

2020-11-07 15:41:57 (1.15 MB/s) - ‘../../data/ml-latest-small.zip’ saved [978202/978202]



In [3]:
import zipfile

with zipfile.ZipFile('../../data/ml-latest-small.zip', 'r') as zip_ref: zip_ref.extractall('../../data')

In [3]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [4]:
links = pd.read_csv('../../data/ml-latest-small/links.csv')

In [5]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [7]:
movies = pd.read_csv('../../data/ml-latest-small/movies.csv')

In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [10]:
ratings = pd.read_csv('../../data/ml-latest-small/ratings.csv')

In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [13]:
tags = pd.read_csv('../../data/ml-latest-small/tags.csv')

In [14]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [15]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [41]:
# import necessary libraries
from pyspark.sql import SparkSession
# instantiate SparkSession object
# spark = SparkSession.builder.master('local').getOrCreate()
spark = SparkSession\
        .builder\
        .appName('ALSExample').config('spark.driver.host', 'localhost')\
        .getOrCreate()
# read in the dataset into pyspark DataFrame
links = spark.read.csv('../../data/ml-latest-small/links.csv', header='true', inferSchema='true')
movies = spark.read.csv('../../data/ml-latest-small/movies.csv', header='true', inferSchema='true')
ratings = spark.read.csv('../../data/ml-latest-small/ratings.csv', header='true', inferSchema='true')
tags = spark.read.csv('../../data/ml-latest-small/tags.csv', header='true', inferSchema='true')

In [16]:
# merged links and movies
movie_ratings = links.merge(movies, on=["movieId"])
movie_ratings.head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [17]:
movie_ratings = ratings.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres
0,1,1,4.0,964982703,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [18]:
movie_ratings = tags.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId_x,movieId,tag,timestamp_x,userId_y,rating,timestamp_y,imdbId,tmdbId,title,genres
0,2,60756,funny,1445714994,2,5.0,1445714980,838283,12133.0,Step Brothers (2008),Comedy
1,2,60756,funny,1445714994,18,3.0,1455749449,838283,12133.0,Step Brothers (2008),Comedy
2,2,60756,funny,1445714994,62,3.5,1528934376,838283,12133.0,Step Brothers (2008),Comedy
3,2,60756,funny,1445714994,68,2.5,1269123243,838283,12133.0,Step Brothers (2008),Comedy
4,2,60756,funny,1445714994,73,4.5,1464196221,838283,12133.0,Step Brothers (2008),Comedy


In [24]:
movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 233212
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userId_x     233213 non-null  int64  
 1   movieId      233213 non-null  int64  
 2   tag          233213 non-null  object 
 3   timestamp_x  233213 non-null  int64  
 4   userId_y     233213 non-null  int64  
 5   rating       233213 non-null  float64
 6   timestamp_y  233213 non-null  int64  
 7   imdbId       233213 non-null  int64  
 8   tmdbId       233213 non-null  float64
 9   title        233213 non-null  object 
 10  genres       233213 non-null  object 
dtypes: float64(2), int64(6), object(3)
memory usage: 21.4+ MB


# Using Matrix Factorization with ALS to Build a Collaborative Filtering Movie Recommendation Model 

In [45]:
# import PySpark, NumPy, IPython

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf, col, when
from pyspark.ml.evaluation import RegressionEvaluator

import numpy as np

from IPython.display import Image
from IPython.display import display

In [46]:
# generate a spark session
spark = SparkSession.builder.appName('FSM').getOrCreate()

In [47]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [48]:
# create spark dataframes

In [49]:
ratings_df = spark.read.csv('../../data/ml-latest-small/ratings.csv', inferSchema=True, header=True)
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [50]:
ratings_df.show(1)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
+------+-------+------+---------+
only showing top 1 row



In [52]:
movies_df = spark.read.csv('../../data/ml-latest-small/movies.csv', inferSchema=True, header=True)
movies_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [53]:
movies_df.show(1)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
+-------+----------------+--------------------+
only showing top 1 row



In [55]:
links_df = spark.read.csv('../../data/ml-latest-small/links.csv', inferSchema=True, header=True)
links_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)



In [56]:
# Split training and testing data
# ALS requires numeric data, our data is already numeric
training_df, validation_df = ratings_df.randomSplit([.8, .2])

In [57]:
# ALS requires the following parameters
iterations = 10
regularization_parameter = 0.1
rank = 4
errors = []
err = 0

In [58]:
# Build the model
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.8840305708681246


In [124]:
# this code checks rmse for ranks 4 - 10
for rank in range(4,10):
    als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank, userCol="userId", itemCol="movieId", ratingCol="rating")
    model = als.fit(training_df)
    predictions = model.transform(validation_df)
    new_predictions = predictions.filter(col('prediction') != np.nan)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(new_predictions)
    print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.8840305708681246
Root-mean_square error = 0.8877305104104398
Root-mean_square error = 0.8837197252045196
Root-mean_square error = 0.8916516039700236
Root-mean_square error = 0.894832573435923
Root-mean_square error = 0.8926116127840896


In [125]:
# change rank to 8, based on above code, this is the highest rmse score
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=8, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.894832573435923


In [ ]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.maxIter, [5, 50, 100, 200]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [129]:
# this code is supposed to  check cross val scores for ranks 4-10 but takes a long time to run and I couldn't
# figure out how to print out the scores

# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# ls = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank, userCol="userId", itemCol="movieId", ratingCol="rating")
# paramGrid = ParamGridBuilder() \
#     .addGrid(als.regParam, [0.1, 0.01, 0.18]) \
#     .addGrid(als.rank, range(4, 10)) \
#     .build()
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# crossval = CrossValidator(estimator=als,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=evaluator,
#                           numFolds=5)
# cvModel = crossval.fit(training_df)

In [65]:
predictions.show(n=5)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   385|    471|   4.0|850766697|  2.889936|
|   602|    471|   4.0|840876085| 3.4433308|
|   603|    471|   4.0|954482443| 3.9306126|
|   555|    471|   3.0|978746933| 3.7236834|
|   136|    471|   4.0|832450058| 4.5717645|
+------+-------+------+---------+----------+
only showing top 5 rows



In [64]:
predictions.join(movies_df, "movieId").select("userId","title","genres","prediction").show(5)

+------+--------------------+------+----------+
|userId|               title|genres|prediction|
+------+--------------------+------+----------+
|   385|Hudsucker Proxy, ...|Comedy|  2.889936|
|   602|Hudsucker Proxy, ...|Comedy| 3.4433308|
|   603|Hudsucker Proxy, ...|Comedy| 3.9306126|
|   555|Hudsucker Proxy, ...|Comedy| 3.7236834|
|   136|Hudsucker Proxy, ...|Comedy| 4.5717645|
+------+--------------------+------+----------+
only showing top 5 rows



In [73]:
for_one_user = predictions.filter(col("userId")==599).join(movies_df, "movieId").join(links_df, "movieId").select("userId","title","genres","tmdbId","prediction")
for_one_user.show(5)

+------+--------------------+--------------------+------+----------+
|userId|               title|              genres|tmdbId|prediction|
+------+--------------------+--------------------+------+----------+
|   599|Just One of the G...|              Comedy| 24548| 2.0350783|
|   599|Tale of Two Siste...|Drama|Horror|Myst...|  4552| 3.6057723|
|   599|Before Sunset (2004)|       Drama|Romance|    80| 2.8824944|
|   599|    Barb Wire (1996)|       Action|Sci-Fi| 11867| 1.9949265|
|   599|Very Bad Things (...|        Comedy|Crime| 10029| 2.5321043|
+------+--------------------+--------------------+------+----------+
only showing top 5 rows



In [74]:
# create code to open web browsers for recommended movies
import webbrowser
link = "https://www.themoviedb.org/movie/"
for movie in for_one_user.take(1):
    movieURL = link+str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Just One of the Guys (1985)


In [105]:
# generate top 5 movie recommendations for each user
userRecomments = model.recommendForAllUsers(5)
# generate top 5 user recommendations for each movie
movieRecomments = model.recommendForAllItems(5)

In [106]:
userRecomments.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [107]:
userRecomments.select("userId","recommendations.movieId").show(5, False)

+------+------------------------------------+
|userId|movieId                             |
+------+------------------------------------+
|471   |[51931, 6818, 306, 1104, 171495]    |
|463   |[104875, 60943, 59018, 6818, 33649] |
|496   |[51931, 158966, 26810, 213, 3030]   |
|148   |[2295, 8235, 299, 89904, 3022]      |
|540   |[33649, 51931, 131724, 179135, 7071]|
+------+------------------------------------+
only showing top 5 rows



In [108]:
movieRecomments.select("movieId","recommendations.UserId").show(5, False)

+-------+-------------------------+
|movieId|UserId                   |
+-------+-------------------------+
|1580   |[53, 543, 43, 93, 337]   |
|4900   |[53, 51, 371, 310, 154]  |
|5300   |[53, 154, 273, 296, 418] |
|6620   |[236, 296, 416, 160, 191]|
|7340   |[12, 578, 224, 594, 548] |
+-------+-------------------------+
only showing top 5 rows



In [109]:
users = ratings_df.select("userId").distinct().limit(3);
users.show()

+------+
|userId|
+------+
|   148|
|   463|
|   471|
+------+



In [110]:
userSubsetRecs = model.recommendForUserSubset(users, 5)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[51931, 4.908990...|
|   463|[[104875, 4.82176...|
|   148|[[2295, 4.6592436...|
+------+--------------------+



In [111]:
userSubsetRecs.select("userId","recommendations.movieId").show(5, False)

+------+-----------------------------------+
|userId|movieId                            |
+------+-----------------------------------+
|471   |[51931, 6818, 306, 1104, 171495]   |
|463   |[104875, 60943, 59018, 6818, 33649]|
|148   |[2295, 8235, 299, 89904, 3022]     |
+------+-----------------------------------+



In [112]:
movies = ratings_df.select("movieId").distinct().limit(3)
movies.show()

+-------+
|movieId|
+-------+
|   1580|
|   2366|
|   3175|
+-------+



In [113]:
movieSubsetRecs = model.recommendForItemSubset(movies, 5)
movieSubsetRecs.select("movieId", "recommendations.UserId").show(5, False)

+-------+------------------------+
|movieId|UserId                  |
+-------+------------------------+
|1580   |[53, 543, 43, 93, 337]  |
|3175   |[53, 543, 43, 93, 77]   |
|2366   |[53, 375, 502, 259, 154]|
+-------+------------------------+



In [120]:
# prediction for some movie for a specific user
movie_ids = [1580,3175,2366,1590]
user_ids = [543,543,543,543]
new_user_preds = sqlContext.createDataFrame(zip(movie_ids, user_ids), schema=['movieId','userId'])

In [121]:
new_predictions = model.transform(new_user_preds)
new_predictions.show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   1580|   543| 4.9348683|
|   2366|   543| 3.1627982|
|   3175|   543|  4.819924|
|   1590|   543|  4.169969|
+-------+------+----------+

